## [다음 영화 예매순위 정보 수집하기](https://movie.daum.net/ranking/reservation)
### 2022.01.28

In [7]:
# 수집한 데이터 저장용
import numpy as np
import pandas as pd
# 요청
import requests
# 수집
import bs4
# 기타
import time
import os
from IPython.display import clear_output

In [8]:
# 요청함수
def getSource(site) :
    # 헤더 정보 셋팅
    header_info = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
    }
    
    # 요청한다
    response = requests.get(site, headers=header_info)
    return response.text.strip()

In [9]:
# 한 페이지의 데이터를 수집해 저장하는 함수
def getData(source, filename) :
    
    # 데이터 있는 전체를 가져온다.
    a1 = source.select_one('#mainContent > div > div.box_ranking > ol')
#     print(a1)
    
    a2 = a1.select('li')
#     print(a2)

    data_dict = {
            '영화제목' : [],
            '평점' : [],
            '예매율' : [],
            '개봉날짜' : [],
            '영화설명' : [],
            '파일명' : []
        }
    
    for movie in a2 :
        
        # 영화 제목
        a3 = movie.select_one('div > div.thumb_cont > strong > a')
        data1 = a3.text.strip()
#         print(data1)

        # 평점 
        a4 = movie.select_one('div > div.thumb_cont > span.txt_append > span:nth-child(1) > span')
        data2 = a4.text.strip()
#         print(data2)

        # 예매율
        a5 = movie.select_one('div > div.thumb_cont > span.txt_append > span:nth-child(2) > span')
        data3 = a5.text.strip()
        data3 = data3[:-1].strip()
#         print(data3)
        
        # 개봉일 
        a6 = movie.select_one('div > div.thumb_cont > span.txt_info > span')
        data4 = a6.text.strip()
#         print(data4)

        # 영화설명
        a7 = movie.select_one('div > div.thumb_item > div.poster_info > a')
        data5 = a7.text.strip()
#         print(data5)
        
        # 저장할 데이터를 담는다.
        data_dict['영화제목'].append(data1)
        data_dict['평점'].append(data2)
        data_dict['예매율'].append(data3)
        data_dict['개봉날짜'].append(data4)
        data_dict['영화설명'].append(data5)
       
    for a8 in data_dict['영화제목']:
        # 파일명 정제
        data6 = fileNameCheck(a8)
        data_dict['파일명'].append(data6)
        
    # 이미지 파일 저장
    a1 = source.select('#mainContent > div > div.box_ranking > ol > li > div > div.thumb_item > div.poster_movie > img')
    for idx, a2 in enumerate(a1) :
        # src 속성 값을 가져온다.
        # print(a2.attrs['src'])
        # 파일 데이터를 받아온다.
        img_res = requests.get(a2.attrs['src'])
        
        # 저장한다.
        with open(f'image/{data_dict["파일명"][idx]}.jpg', 'wb') as fp :
            fp.write(img_res.content)

# 데이터 프레임을 생성한다.
    df1 = pd.DataFrame(data_dict)
    # display(df1)
    
    # 저장한다.
    # 파일이 없다면 : 첫 번째 저장
    if os.path.exists(fileName) == False :
        df1.to_csv(fileName, encoding='utf-8-sig', index=False)
    # 파일이 있다면..
    else :
        df1.to_csv(fileName, encoding='utf-8-sig', index=False, mode='a', header=None)

In [10]:
# 다음 페이지 주소를 가져오는 함수
def getNextPage(source) :
    pass

In [11]:
# 파일명 정제
def fileNameCheck(fileName) :
    # 윈도우에서 허용하지 않는 문자는 모두 제외한다.
    fileName = fileName.replace('₩₩', ' ')
    fileName = fileName.replace('/', ' ')
    fileName = fileName.replace(':', ' ')
    fileName = fileName.replace('*', ' ')
    fileName = fileName.replace('?', ' ')
    fileName = fileName.replace('"', ' ')
    fileName = fileName.replace('<', ' ')
    fileName = fileName.replace('>', ' ')
    fileName = fileName.replace('|', ' ')
    
    return fileName

In [13]:
# 페이지의 주소
site = 'https://movie.daum.net/ranking/reservation'
nextPage = ''
fileName = 'data3.csv'

# 파일이 존재하면 삭제한다.
if os.path.exists(fileName) == True :
    os.remove(fileName)
    
# 서버에 접속해서 응답 결과를 가져온다.
a1 = getSource(f'{site}{nextPage}')
source = bs4.BeautifulSoup(a1)

# source

# 현재 페이지의 데이터를 수집한다.
getData(source, fileName)

print('수집 완료')

수집 완료
